In [21]:
import os
import gc
import sys
import argparse
import yaml
import time
import shutil

import pandas as pd
import polars as pl


In [22]:
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/data")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/submission")


import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from dss_dataloader import get_loader
from dss_model import get_model
from training_loop import concat_valid_input_info, get_valid_values_dict


In [23]:
exp_dir = "/kaggle/working"
exp_name = "exp020_dense_chh_skffold_epoch30" # exp020_dense_chh_skffold_epoch30からbest modelを保存するようになった
# series_df_path = "/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet"
series_df_path = "/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet"
tmp_file_path = "/kaggle/working/tmp_event"
split_file_path = "/kaggle/working/tmp"
os.makedirs(tmp_file_path, exist_ok=True)
config_path = os.path.join(exp_dir, exp_name, "config.yaml")

class_pred_roll_mean_num = 9
class_pred_maxpool_num = 3

dataframe_split_num = 10


# preprocess

In [24]:
def pl_datetime_preprocess(train_series_):
    train_series_ = train_series_.with_columns(
        pl.col("timestamp").str.to_datetime().dt.replace_time_zone(None))
    train_series_ = train_series_.with_columns(
        pl.col("timestamp").dt.second().cast(pl.Int32).alias("second"))
    train_series_ = train_series_.with_columns(
        pl.col("timestamp").dt.minute().cast(pl.Int32).alias("minute"))
    train_series_ = train_series_.with_columns(
        pl.col("timestamp").dt.date().cast(str).alias("date"))
    return train_series_


def preprocess_input(train_series_: pd.DataFrame) -> pd.DataFrame:
    train_series_ = train_series_.drop(columns=["timestamp"], axis=1)
    # anglezとenmoのrolling meanとrolling stdを取る
    print("get anglez and enmo rolling mean and std")
    for roll_num in [36, 60]:  # 雰囲気で選んだ
        train_series_[f"anglez_mean_{roll_num}"] = (
            train_series_.groupby("series_id")["anglez"].rolling(
                roll_num, center=True).mean().reset_index(0, drop=True))
        train_series_[f"enmo_mean_{roll_num}"] = (
            train_series_.groupby("series_id")["enmo"].rolling(
                roll_num, center=True).mean().reset_index(0, drop=True))
        train_series_[f"anglez_std_{roll_num}"] = (
            train_series_.groupby("series_id")["anglez"].rolling(
                roll_num, center=True).std().reset_index(0, drop=True))
        train_series_[f"enmo_std_{roll_num}"] = (
            train_series_.groupby("series_id")["enmo"].rolling(
                roll_num, center=True).std().reset_index(0, drop=True))
        train_series_[f"anglez_mean_{roll_num}"] = train_series_[
            f"anglez_mean_{roll_num}"].fillna(0)
        train_series_[f"enmo_mean_{roll_num}"] = train_series_[
            f"enmo_mean_{roll_num}"].fillna(0)
        train_series_[f"anglez_std_{roll_num}"] = train_series_[
            f"anglez_std_{roll_num}"].fillna(0)
        train_series_[f"enmo_std_{roll_num}"] = train_series_[
            f"enmo_std_{roll_num}"].fillna(0)
    
    return train_series_


def set_seriesdatekey(train_series_: pd.DataFrame) -> pd.DataFrame:
    train_series_["series_date_key"] = (
        train_series_["series_id"].astype(str) + "_" + train_series_["date"].astype(str)
    )
    return train_series_

def label_encode_series_date_key(train_series_: pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import LabelEncoder

    le = LabelEncoder()
    train_series_["series_date_key_str"] = train_series_["series_date_key"].astype(str)
    train_series_["series_date_key"] = le.fit_transform(
        train_series_["series_date_key_str"]
    )
    train_series_["series_date_key"] = train_series_["series_date_key"].astype("int64")
    return train_series_


# post process

In [25]:
# 1step 0.5secで30minなら60*30=1800step?
# metric的にいっぱい検出してもいい？とりあえず小さめ
def detect_event_from_downsample_classpred(CFG, df, N=class_pred_roll_mean_num, maxpool_kernel_size=class_pred_maxpool_num, maxpool_stride=1):
    df = df[df["second"] == 0].reset_index(drop=True)
    for fold in CFG.folds:
        df["class_pred_beforemean"] = df.groupby("series_id")[f"class_pred_fold{fold}"].apply(
            lambda x: x.rolling(N, min_periods=1).mean()
        ).reset_index(drop=True)
        df["class_pred_aftermean"] = df.groupby("series_id")[f"class_pred_fold{fold}"].apply(
            lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
        ).reset_index(drop=True)

        df[f"event_pred_fold{fold}"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
        # 入力サイズと出力サイズが一致するようにpaddingを調整
        maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
        # maxpoolしてピーク検出
        max_pooling = nn.MaxPool1d(maxpool_kernel_size,
                                   stride=maxpool_stride,
                                   padding=maxpool_padding)
        event_pred = df[f"event_pred_fold{fold}"].values
        event_pred = torch.tensor(event_pred).unsqueeze(0)
        pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
        event_pred = event_pred.squeeze(0).numpy()
        # peakのところだけ残すmaskを作成
        peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
        peak_event_pred = event_pred * peak_event_pred_mask
        df[f"event_pred_fold{fold}"] = peak_event_pred
        df[f"onset_pred_fold{fold}"] = np.clip(-df[f"event_pred_fold{fold}"], 0, 1)
        df[f"wakeup_pred_fold{fold}"] = np.clip(df[f"event_pred_fold{fold}"], 0, 1)
        df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df


def make_submission_df(df, threshold=0.01):
    df = df[["series_id", "step", "event_pred"]]
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(
        lambda x: 1 if x > threshold else -1 if x < -threshold else 0
    )
    df = df[df["event"] != 0]
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    df = df.drop("event_pred", axis=1)
    df = df.reset_index(drop=True)
    return df


# infer

## data utils

In [26]:
def get_pred_df(
    input_info_dict: dict,
    preds_dict: dict,
    pred_df: pd.DataFrame,
    fold: int,
) -> pd.DataFrame:
    start_time = time.time()
    print("creating oof_df", end=" ... ")
    if "class_pred" in pred_df.columns:
        pred_df = pred_df.drop(["class_pred"], axis=1)
    series_date_key_list = []
    class_pred_list, steps_list = [],  []

    for idx, (series_date_key, start_step, end_step) in enumerate(
            zip(
                input_info_dict["series_date_key"],
                input_info_dict["start_step"],
                input_info_dict["end_step"],
            )):
        if not isinstance(series_date_key, np.int64):
            series_date_key = series_date_key.numpy()
        # preds targets shape: [batch, ch, data_length]
        class_pred = preds_dict["class_preds"][idx]
        steps = range(start_step, end_step + 1, 12)
        series_date_data_num = len(steps)
        if series_date_data_num < len(class_pred[0]):
            class_pred = class_pred[0, :series_date_data_num]
        elif series_date_data_num > len(class_pred[0]):
            padding_num = series_date_data_num - len(class_pred[0])
            class_pred = np.concatenate(
                [class_pred[0], -1 * np.ones(padding_num)], axis=0)
        else:
            class_pred = class_pred[0]
        if not (len(class_pred) == len(steps)):
            print("len(class_pred)", len(class_pred))
            print("len(steps)", len(steps))
            raise ValueError("preds and step length is not same")
        class_pred_list.extend(class_pred)
        steps_list.extend(steps)
        series_date_key_list.extend([series_date_key] * len(steps))
    pred_col_name = f"class_pred_fold{fold}"
    oof_pred_target_df = pd.DataFrame({
        "series_date_key": series_date_key_list,
        "step": steps_list,
        pred_col_name: class_pred_list,
    })
    merge_start_time = time.time()
    print("merging oof_df")
    oof_pred_target_df["series_date_key"] = oof_pred_target_df["series_date_key"].astype("int64")
    pred_df = pd.merge(pred_df,
                       oof_pred_target_df,
                       on=["series_date_key", "step"],
                       how="left")
    pred_df[pred_col_name] = pred_df[pred_col_name].fillna(0)
    merge_elapsed = int(time.time() - merge_start_time) / 60
    print("merge elapsed time: {:.2f} min".format(merge_elapsed))
    elapsed = int(time.time() - start_time) / 60
    print(f" >> oof_df created. elapsed time: {elapsed:.2f} min")
    return pred_df        


## pred & infer

In [27]:
def predict(CFG, model, infer_loader):
    model.eval()

    infer_predictions = {"class_preds": np.empty(0)}
    infer_input_info = {"series_date_key": [], "start_step": [], "end_step": []}

    for _, (inputs, input_info_dict) in enumerate(infer_loader):
        inputs = inputs.to(CFG.device, non_blocking=True).float()
        with torch.no_grad():
            preds = model(inputs)

        infer_predictions = get_valid_values_dict(
            preds, infer_predictions, mode="preds"
        )
        infer_input_info = concat_valid_input_info(infer_input_info, input_info_dict)

    del inputs, preds
    gc.collect()
    torch.cuda.empty_cache()
    return infer_predictions, infer_input_info


def infer_onedf(CFG, series_df, split_idx):
    series_df = pl_datetime_preprocess(series_df)
    series_df = series_df.to_pandas()
    series_df = preprocess_input(series_df)
    series_df = set_seriesdatekey(series_df)
    series_df = label_encode_series_date_key(series_df)
    key_df = series_df[["series_date_key", "series_date_key_str"]].drop_duplicates()
    key_df = key_df.reset_index(drop=True)


    key_df["series_id"], key_df["date"] = key_df["series_date_key_str"].str.split("_", expand=True)
    key_df = key_df.drop(columns=["series_date_key_str"], axis=1)

    pred_df = series_df[["series_id", "series_date_key", "step", "second"]].copy()
    for fold in CFG.folds:
        print(f"-- fold{fold} inference start --")
        # set model & learning fn
        model = get_model(CFG)
        model_path = os.path.join(exp_dir, exp_name, f"fold{fold}_model.pth")
        print("model loading", model_path)
        model.load_state_dict(torch.load(model_path))
        model = model.to(CFG.device)
        # separate train/valid data
        infer_loader = get_loader(CFG, key_df, series_df, mode="test")
        infer_preds, infer_input_dict = predict(CFG, model, infer_loader)
        print(f"split[{split_idx}] fold[{fold}] prediction finished.")
        pred_df = get_pred_df(
            infer_input_dict,
            infer_preds,
            pred_df,
            fold,
        )
        del infer_preds, infer_input_dict, infer_loader, model
        gc.collect()
        torch.cuda.empty_cache()

    # meanでいいの？maxにしてみる？
    pred_df = detect_event_from_downsample_classpred(CFG, pred_df)
    # mean? max?
    pred_df["event_pred"] = pred_df[[f"event_pred_fold{fold}" for fold in CFG.folds]].mean(axis=1)
    # pred_df["event_pred"] = pred_df[[f"event_pred_fold{fold}" for fold in CFG.folds]].max(axis=1)
    pred_df = pred_df.drop(columns=[f"event_pred_fold{fold}" for fold in CFG.folds])
#     display(pred_df)
    pred_df.to_csv(os.path.join(tmp_file_path, f"pred_df_split_{split_idx}.csv"))
    return pred_df
    
def inference(
    CFG, exp_dir, exp_name, series_df_path, tmp_file_path, split_num=3
):
    infer_start_time = time.time()
    infer_data_num = -1
    for idx in range(split_num):
        print("split idx:", idx)
        series_df = pl.read_parquet(
            os.path.join(split_file_path, f"series_df_split_{idx}.parquet")
        )
        # testが見えているものを読み込んだときでもむりやり動かす
        if infer_data_num==450:
            series_df = pl.read_parquet(series_df_path)
            pred_df = infer_onedf(CFG, series_df, idx)
            sub_df_split = make_submission_df(pred_df)
            sub_df_split_path = os.path.join(tmp_file_path, f"sub_df.csv")
            sub_df_split.to_csv(sub_df_split_path, index=False)
            break
        pred_df = infer_onedf(CFG, series_df, idx)
        
        sub_df_split = make_submission_df(pred_df)
        sub_df_split_path = os.path.join(tmp_file_path, f"sub_df_split_{idx}.csv")
        sub_df_split.to_csv(sub_df_split_path, index=False)
        print(f"sub_df_split is saved as {sub_df_split_path}")
        del sub_df_split, pred_df, series_df
        gc.collect()
        torch.cuda.empty_cache()

    # testが見えているものを読み込んだときでもむりやり動かす
    if infer_data_num==450:
        sub_df = pd.read_csv(sub_df_split_path)
        sub_df_tmp = None
    else:
        sub_df = pd.DataFrame()
        for idx in range(split_num):
            if idx==0:
                sub_df = pd.read_csv(os.path.join(tmp_file_path, f"sub_df_split_{idx}.csv"))
            else:
                sub_df_tmp = pd.read_csv(os.path.join(tmp_file_path, f"sub_df_split_{idx}.csv"))
                if len(sub_df_tmp) > 0:
                    sub_df = pd.concat([sub_df, sub_df_tmp], axis=0)       

    del sub_df_tmp
    gc.collect()
    torch.cuda.empty_cache()
    sub_df = sub_df.reset_index(drop=True)
    print(f"sub_df data num : {len(sub_df)}")
    return sub_df


In [28]:
config = yaml.load(open(config_path, "r"), Loader=yaml.SafeLoader)
config = argparse.Namespace(**config)

# fold 0 だけにしてみる
# config.folds = [0]

print(config)
print("--")
print("infer start")
sub_df = inference(config, exp_dir, exp_name, series_df_path, tmp_file_path, split_num=dataframe_split_num)


Namespace(T_0=30, T_mult=1, ave_kernel_size=301, batch_size=64, class_loss_weight=1.0, class_output_channels=1, competition_dir='/kaggle/input/child-mind-institute-detect-sleep-states', competition_name='dss', device='cuda', embedding_base_channels=16, eta_min=1e-09, event_df='/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv', event_loss_weight=1.0, event_output_channels=2, exp_category='earlysave', exp_dir='/kaggle/working/exp020_dense_chh_skffold_epoch30', exp_name='exp020_dense_chh_skffold_epoch30', folds=[0, 1, 2, 3, 4], group_key='series_id', input_channels=6, input_dir='/kaggle/input', key_df='/kaggle/input/datakey_unique_non_null.csv', logger_path='/kaggle/working/exp020_dense_chh_skffold_epoch30/train.log', lr=0.001, maxpool_kernel_size=11, model_type='input_target_downsample_dense', n_epoch=30, n_folds=5, num_workers=2, output_channels=2, output_dir='/kaggle/working', print_freq=50, pseudo_weight_exp='exp003', seed=42, series_df='/kaggle/input/targetdown

get anglez and enmo rolling mean and std
-- fold0 inference start --
model type =  input_target_downsample_dense
model loading /kaggle/working/exp020_dense_chh_skffold_epoch30/fold0_model.pth
split[0] fold[0] prediction finished.
creating oof_df ... merging oof_df
merge elapsed time: 0.03 min
 >> oof_df created. elapsed time: 0.05 min
-- fold1 inference start --
model type =  input_target_downsample_dense
model loading /kaggle/working/exp020_dense_chh_skffold_epoch30/fold1_model.pth
split[0] fold[1] prediction finished.
creating oof_df ... merging oof_df
merge elapsed time: 0.03 min
 >> oof_df created. elapsed time: 0.05 min
-- fold2 inference start --
model type =  input_target_downsample_dense
model loading /kaggle/working/exp020_dense_chh_skffold_epoch30/fold2_model.pth
split[0] fold[2] prediction finished.
creating oof_df ... merging oof_df
merge elapsed time: 0.03 min
 >> oof_df created. elapsed time: 0.05 min
-- fold3 inference start --
model type =  input_target_downsample_dense

In [29]:
sub_df["row_id"] = range(len(sub_df))
sub_df = sub_df[["row_id", "series_id", "step", "event", "score"]]
if len(sub_df) == 0:
    sub_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv")
# sub_df.to_csv("submission.csv", index=False)
display(sub_df)


,row_id,series_id,step,event,score
0,0,038441c925bb,0,wakeup,0.057968
1,1,038441c925bb,612,wakeup,0.025244
2,2,038441c925bb,636,wakeup,0.017577
3,3,038441c925bb,2352,onset,0.012584
4,4,038441c925bb,2856,wakeup,0.010292
...,...,...,...,...,...
92281,92281,fe90110788d2,591708,onset,0.050172
92282,92282,fe90110788d2,591828,wakeup,0.024425
92283,92283,fe90110788d2,591996,wakeup,0.017652
92284,92284,fe90110788d2,592116,wakeup,0.027364


In [30]:
from dss_metrics import score
check_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
check_event_df = check_event_df[check_event_df["series_id"].isin(sub_df["series_id"].unique())].dropna().reset_index(drop=True)
print(score(check_event_df, sub_df))


0.7195194264754544
